In [6]:
import netpixi
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('reviews_april.csv', sep=',')

In [8]:
df.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,68330832,brazilian,"Jogo excelente em todas as mecânicas, gráficos...",2020-04-30 23:52:54,2020-04-30 23:52:54,True,0,0,...,True,False,False,76561198159239276,26,2,9640.0,0.0,9009.0,1.588377e+09
1,292030,The Witcher 3: Wild Hunt,68329511,brazilian,o jogo é pika,2020-04-30 23:19:24,2020-04-30 23:19:24,True,0,0,...,True,False,False,76561198043602884,34,6,10299.0,0.0,2568.0,1.606605e+09
2,292030,The Witcher 3: Wild Hunt,68328796,brazilian,Nice,2020-04-30 23:01:10,2020-04-30 23:01:10,True,0,0,...,True,False,False,76561198134272383,21,5,783.0,0.0,619.0,1.609530e+09
3,292030,The Witcher 3: Wild Hunt,68327598,brazilian,muito bom,2020-04-30 22:30:41,2020-04-30 22:30:41,True,0,0,...,True,False,False,76561199006967739,12,2,5922.0,0.0,5533.0,1.590269e+09
4,292030,The Witcher 3: Wild Hunt,68325545,brazilian,"Temática e história ricas de detalhes, um bom ...",2020-04-30 21:42:55,2020-04-30 21:42:55,True,0,0,...,True,False,False,76561198062670500,36,1,5192.0,0.0,1514.0,1.589417e+09


In [9]:
def get_edges(df):
    users = df['author.steamid'].unique()
    
    edges = []
    for user in users:
        games = df[df['author.steamid']==user]['app_name'].unique()
        for game in games:
            users_game = df[df['app_name']==game]['author.steamid'].unique()
            for user_game in users_game:
                if user_game != user:
                    edges.append(tuple(sorted([user, user_game])))
    
    edges = np.array(edges)

    edges, counts = np.unique(edges, axis=0, return_counts=True)

    mean = np.mean(counts)

    edges = edges[counts>mean]
    
    
    return edges

In [61]:
edges = get_edges(df)
edges

array([[76561197960544484, 76561198098119486],
       [76561197960544484, 76561198131459117],
       [76561197960544484, 76561198168409304],
       ...,
       [76561199026950361, 76561199030327698],
       [76561199029691049, 76561199046189961],
       [76561199030327698, 76561199039810592]])

In [90]:
# save edges in json file
with open('edges.json', 'w') as f:
    json.dump(edges.tolist(), f)

In [10]:
with open('edges.json', 'r') as f:
    edges = json.load(f)

In [11]:
from netpixi.integration.gt import Graph

g = Graph(directed=False) # não-dirigido

In [12]:
for edge in edges:
    u = int(edge[0])
    v = int(edge[1])
    if not g.has_vertex(u):
        g.add_vertex(u)

    if not g.has_vertex(v):
        g.add_vertex(v)

    if not g.has_edge(u, v):
        g.add_edge(u, v)

In [13]:
u = g.get_vertex(76561197960544484)
u.get_id()

76561197960544484

In [14]:
print('número de arestas de g:', g.num_edges())

número de arestas de g: 19129


In [24]:
import netpixi
from netpixi.integration.gt import *

In [19]:
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)

In [25]:
gt_save(g, 'reviews_april.net.gz')

In [27]:
r = netpixi.render('reviews_april.net.gz')

In [22]:
r.vertex_default(color=0xD420E9, size=1)

In [23]:
r.edge_default(color=0xFF1401,width=3,curve1=1,curve2=1)